Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'
#pickle_file = 'notMNIST_sanitize.pickle'
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels  = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels  = save['valid_labels']
    test_dataset  = save['test_dataset']
    test_labels   = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 256
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset  = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases  = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases  = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases  = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(      [num_hidden, num_labels], stddev=0.1))
    layer4_biases  = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv    = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden  = tf.nn.relu(conv + layer1_biases)
        conv    = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden  = tf.nn.relu(conv + layer2_biases)
        shape   = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden  = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss   = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction  = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 50001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 2000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.286114
Minibatch accuracy: 9.0%
Validation accuracy: 10.0%
Minibatch loss at step 2000: 0.400900
Minibatch accuracy: 89.1%
Validation accuracy: 86.6%
Minibatch loss at step 4000: 0.470911
Minibatch accuracy: 85.9%
Validation accuracy: 88.5%
Minibatch loss at step 6000: 0.356559
Minibatch accuracy: 89.8%
Validation accuracy: 89.1%
Minibatch loss at step 8000: 0.350917
Minibatch accuracy: 90.2%
Validation accuracy: 89.3%
Minibatch loss at step 10000: 0.350548
Minibatch accuracy: 89.5%
Validation accuracy: 89.6%
Minibatch loss at step 12000: 0.276603
Minibatch accuracy: 91.4%
Validation accuracy: 89.8%
Minibatch loss at step 14000: 0.262403
Minibatch accuracy: 91.8%
Validation accuracy: 90.1%
Minibatch loss at step 16000: 0.337457
Minibatch accuracy: 90.6%
Validation accuracy: 90.4%
Minibatch loss at step 18000: 0.342341
Minibatch accuracy: 87.5%
Validation accuracy: 90.4%
Minibatch loss at step 20000: 0.283443
Minibatch accuracy: 92.2%
Validation a

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
batch_size = 256
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset  = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases  = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases  = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases  = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(      [num_hidden, num_labels], stddev=0.1))
    layer4_biases  = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):   
        # Layer1: convolution, rectifier, and pooling
        conv   = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding = 'SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        pool   = tf.nn.max_pool(hidden, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
        
        # Layer2: convolution, rectifier, and pooling
        conv   = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding = 'SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        pool   = tf.nn.max_pool(hidden, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
        
        # Layer3: fully connected
        shape   = pool.get_shape().as_list() #print(shape)
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden  = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss   = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction  = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 50001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 2000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.368519
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 2000: 0.408678
Minibatch accuracy: 88.7%
Validation accuracy: 88.4%
Minibatch loss at step 4000: 0.408821
Minibatch accuracy: 86.3%
Validation accuracy: 89.6%
Minibatch loss at step 6000: 0.315644
Minibatch accuracy: 91.0%
Validation accuracy: 90.0%
Minibatch loss at step 8000: 0.295319
Minibatch accuracy: 91.0%
Validation accuracy: 90.5%
Minibatch loss at step 10000: 0.350314
Minibatch accuracy: 88.7%
Validation accuracy: 90.9%
Minibatch loss at step 12000: 0.224129
Minibatch accuracy: 93.4%
Validation accuracy: 90.9%
Minibatch loss at step 14000: 0.259192
Minibatch accuracy: 93.0%
Validation accuracy: 91.3%
Minibatch loss at step 16000: 0.280591
Minibatch accuracy: 93.8%
Validation accuracy: 91.3%
Minibatch loss at step 18000: 0.302135
Minibatch accuracy: 91.0%
Validation accuracy: 91.4%
Minibatch loss at step 20000: 0.262379
Minibatch accuracy: 90.6%
Validation a

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [9]:
batch_size = 256
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset  = tf.constant(test_dataset)
    beta             = tf.placeholder(tf.float32)
    
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases  = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases  = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases  = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(      [num_hidden, num_labels], stddev=0.1))
    layer4_biases  = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    global_step    = tf.Variable(0)
    
    # Model.
    def model(data,train=False):   
        # Layer1: convolution, rectifier, and pooling
        conv   = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding = 'SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        pool   = tf.nn.max_pool(hidden, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
        
        # Layer2: convolution, rectifier, and pooling
        conv   = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding = 'SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        pool   = tf.nn.max_pool(hidden, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
        
        # Layer3: fully connected
        shape   = pool.get_shape().as_list() #print(shape)
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden  = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
        # Dropout
        if (train):           
            hidden = tf.nn.dropout(hidden, 0.5)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    
    # Regularization
    Regularizer = beta * (tf.nn.l2_loss(layer1_weights) +
                          tf.nn.l2_loss(layer2_weights) +
                          tf.nn.l2_loss(layer3_weights) +
                          tf.nn.l2_loss(layer4_weights))
    
    loss   = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + Regularizer

    # Optimizer.
    learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.85, staircase = True)
    optimizer     = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction  = tf.nn.softmax(model(tf_test_dataset))

In [10]:
num_steps = 50001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 1e-2}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 2000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6.038127
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 2000: 0.885382
Minibatch accuracy: 87.9%
Validation accuracy: 87.9%
Minibatch loss at step 4000: 0.697750
Minibatch accuracy: 83.6%
Validation accuracy: 88.5%
Minibatch loss at step 6000: 0.535705
Minibatch accuracy: 89.5%
Validation accuracy: 89.4%
Minibatch loss at step 8000: 0.552227
Minibatch accuracy: 89.1%
Validation accuracy: 89.5%
Minibatch loss at step 10000: 0.542892
Minibatch accuracy: 87.9%
Validation accuracy: 89.7%
Minibatch loss at step 12000: 0.452546
Minibatch accuracy: 92.2%
Validation accuracy: 89.7%
Minibatch loss at step 14000: 0.454428
Minibatch accuracy: 92.2%
Validation accuracy: 89.9%
Minibatch loss at step 16000: 0.529583
Minibatch accuracy: 89.1%
Validation accuracy: 89.9%
Minibatch loss at step 18000: 0.573427
Minibatch accuracy: 86.7%
Validation accuracy: 89.9%
Minibatch loss at step 20000: 0.513395
Minibatch accuracy: 89.5%
Validation a

In [11]:
#########
### 2 ###
#########

batch_size = 128
patch_size = 5
depth1 = 16
depth2 = 32
num_hidden1 = 128
num_hidden2 = 64

# Construct a 7-layer CNN.
# C1: convolutional layer, batch_size x 28 x 28 x 16, convolution size: 5 x 5 x 1 x 16
# S2: sub-sampling layer, batch_size x 14 x 14 x 16
# C3: convolutional layer, batch_size x 10 x 10 x 32, convolution size: 5 x 5 x 6 x 32
# S4: sub-sampling layer, batch_size x 5 x 5 x 32
# C5: convolutional layer, batch_size x 1 x 1 x 128, convolution size: 5 x 5 x 16 x 128
# F6: fully-connected layer, weight size: 128 x 64
# Output layer, weight size: 64 x 10

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset  = tf.constant(test_dataset)
    beta             = tf.placeholder(tf.float32)
    
    # Variables.    
    layer1_weights = tf.Variable(tf.truncated_normal(      [patch_size, patch_size, num_channels, depth1], stddev=0.1))
    layer1_biases  = tf.Variable(tf.zeros([depth1]))
    layer2_weights = tf.Variable(tf.truncated_normal(      [patch_size, patch_size, depth1,       depth2], stddev=0.1))
    layer2_biases  = tf.Variable(tf.constant(1.0, shape=[depth2]))
    layer3_weights = tf.Variable(tf.truncated_normal(      [patch_size, patch_size, depth2, num_hidden1],  stddev=0.1))
    layer3_biases  = tf.Variable(tf.constant(1.0, shape=[num_hidden1]))
    layer4_weights = tf.Variable(tf.truncated_normal(      [num_hidden1, num_hidden2], stddev=0.1))
    layer4_biases  = tf.Variable(tf.constant(1.0, shape=[num_hidden2]))
    layer5_weights = tf.Variable(tf.truncated_normal(      [num_hidden2, num_labels], stddev=0.1))
    layer5_biases  = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    global_step    = tf.Variable(0)
    
    # Model.
    def model(data,train=True):   
       
        # Layer1: convolution, rectifier, and pooling
        conv = tf.nn.conv2d(data, layer1_weights, [1,1,1,1], 'SAME', name='C1')
        hidden = tf.nn.relu(conv + layer1_biases)
        sub = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], 'SAME', name='S2')
        
        # Layer2: convolution, rectifier, and pooling
        conv = tf.nn.conv2d(sub, layer2_weights, [1,1,1,1], padding='VALID', name='C3')
        hidden = tf.nn.relu(conv + layer2_biases)
        sub = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], 'SAME', name='S4')
        
        # Layer3: convolution, rectifier
        conv = tf.nn.conv2d(sub, layer3_weights, [1,1,1,1], padding='VALID', name='C5')
        hidden = tf.nn.relu(conv + layer3_biases)
        
        # Layer4: fully connected
        shape   = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden  = tf.nn.relu(tf.matmul(reshape, layer4_weights) + layer4_biases)
      
        # Dropout
        if (train):           
            hidden = tf.nn.dropout(hidden, 0.5)
        return tf.matmul(hidden, layer5_weights) + layer5_biases

    # Training computation.
    logits = model(tf_train_dataset)
    
    # Regularization
    Regularizer = beta * (tf.nn.l2_loss(layer1_weights) +
                          tf.nn.l2_loss(layer2_weights) +
                          tf.nn.l2_loss(layer3_weights) +
                          tf.nn.l2_loss(layer4_weights))
    
    loss   = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + Regularizer

    # Optimizer.
    learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.85, staircase = True)
    optimizer     = tf.train.AdagradOptimizer(learning_rate).minimize(loss, global_step = global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction  = tf.nn.softmax(model(tf_test_dataset))

In [12]:
num_steps = 50001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 1e-2}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 2000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 9.103950
Minibatch accuracy: 7.0%
Validation accuracy: 10.1%
Minibatch loss at step 2000: 0.662356
Minibatch accuracy: 85.2%
Validation accuracy: 87.0%
Minibatch loss at step 4000: 0.625519
Minibatch accuracy: 89.1%
Validation accuracy: 88.0%
Minibatch loss at step 6000: 0.564198
Minibatch accuracy: 86.7%
Validation accuracy: 88.3%
Minibatch loss at step 8000: 0.496623
Minibatch accuracy: 90.6%
Validation accuracy: 88.7%
Minibatch loss at step 10000: 0.599976
Minibatch accuracy: 85.9%
Validation accuracy: 88.8%
Minibatch loss at step 12000: 0.506531
Minibatch accuracy: 92.2%
Validation accuracy: 88.7%
Minibatch loss at step 14000: 0.581041
Minibatch accuracy: 87.5%
Validation accuracy: 89.2%
Minibatch loss at step 16000: 0.494303
Minibatch accuracy: 89.1%
Validation accuracy: 89.2%
Minibatch loss at step 18000: 0.548238
Minibatch accuracy: 86.7%
Validation accuracy: 89.5%
Minibatch loss at step 20000: 0.500488
Minibatch accuracy: 89.8%
Validation a